In [1]:
!pip install -r requirements.txt

In [2]:
import json
import sys
import os
sys.path.append("src")

In [3]:
%load_ext autoreload
%autoreload 2

In [7]:
%reload_ext autoreload

In [8]:
from wiki_extract import Extracter
from wiki_gen import WikiGen

In [6]:
def get_article(name):
    print('Article name: ', name)
    page = Extracter(name)
    page.get_references()
    page.get_outline()
    page.get_reference_positions()
    page.newspaper_ref()
    page.get_filtered_outline()
    return page

In [18]:
article_names = [
    'Python',
    'Летние Олимпийские игры 2024',
    'Квантовый компьютер',
    'Присоединение Крыма к Российской Федерации',
    'Сколково (инновационный центр)',
    'Tomb Raider (игра, 2013)',
    'Чёрная дыра',
    'Экономика США',
    'Искусственный интеллект',
    'COVID-19',
    'Броненосный крейсер',
    'Применение искусственного интеллекта',
    'РИА Новости'
]

In [8]:
def save_texts(article_name, texts, number):

    directory = os.path.join("Articles", "Sources", article_name)

    os.makedirs(directory, exist_ok=True)
    '''
    for i, text in enumerate(texts, start=number):
        file_path = os.path.join(directory, f"source_{i}.txt")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text)
    '''

In [9]:
articles = {}

for name in article_names:
    page = get_article(name)
    articles[name] = page.ref_texts.keys()
    num = 1
    for _, texts in page.ref_texts.items():
        save_texts(page.name, texts, num)
        num += len(texts)

with open('saved_ref.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=2)

Article name:  COVID-19


Calculating reference positions: 100%|██████████| 57/57 [00:00<00:00, 57332.21it/s]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Calculating reference positions: 100%|██████████| 57/57 [00:00<00:00, 56963.39it/s]


Article name:  Броненосный крейсер


Calculating reference positions: 100%|██████████| 31/31 [00:00<00:00, 30994.86it/s]


Article name:  Применение искусственного интеллекта


Calculating reference positions: 100%|██████████| 48/48 [00:00<00:00, 48049.31it/s]


Article name:  РИА Новости


Calculating reference positions: 100%|██████████| 31/31 [00:00<?, ?it/s]


In [30]:
oclient = OpenAI(api_key=KEY, base_url=URL)

wiki_writer = WikiGen(oclient)

for name in article_names:

    json_str = wiki_writer.get_subqueries(name)

    if json_str:
        data = json.loads(json_str)
        directory = os.path.join("Generation", "Subqueries", name)
        os.makedirs(directory, exist_ok=True)
        filename = name.replace(" ", "_") + '.json'
        file_path = os.path.join(directory, filename)
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

In [22]:
from openai import AsyncOpenAI
from openai_utils import LlmCompleter

client = LlmCompleter(api_address=URL, api_key=KEY)
wiki_writer = WikiGen(client)


for name in article_names:
    topic = name
    texts = []
    file_names = []
    directory = 'Articles/Sources/' + name
    for file in os.listdir(directory):
        if file.endswith('.txt'):
            file_path = os.path.join(directory, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                texts.append(f.read())
            file_names.append(file)
    
    lst = await wiki_writer.filter_sources(topic, "", texts)
    #print(lst)
    c = 0
    for elem in lst:
        if elem[1] > 0.5:
            c += 1
    print(c / len(lst))

1.0
1.0
0.8214285714285714
0.9090909090909091
0.9325153374233128
0.8518518518518519
0.972972972972973
1.0
0.967741935483871


C:\Users\dagri\anaconda3\envs\WIKIbench\Lib\typing.py:430: RuntimeWarning: coroutine 'WikiGen.process_text' was never awaited
  ev_args = tuple(


1.0
1.0
0.6666666666666666
0.8873239436619719


In [28]:
print(await wiki_writer.get_subqueries('COVID-19'))

{
  "article_title": "COVID-19",
  "subqueries": [
    "Каковы основные пути передачи вируса SARS-CoV-2 и какие меры предотвращения наиболее эффективны?",
    "Какие симптомы и клинические проявления характерны для COVID-19, и как они различаются у разных групп населения?",
    "Какие методы диагностики COVID-19 существуют, и какие из них наиболее точны и доступны?",
    "Какие вакцины против COVID-19 были разработаны, и каковы их эффективность и побочные эффекты?",
    "Какие экономические последствия вызвал пандемический кризис, и какие меры поддержки были приняты правительствами?",
    "Как пандемия COVID-19 повлияла на образование, работу и социальные взаимодействия?",
    "Какие научные исследования проводятся для изучения долгосрочных последствий COVID-19 для здоровья?"
  ]
}
